In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sys import path
%matplotlib inline

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')
%cd /content/drive/MyDrive/PHI/ToF_ML/src

/content/drive/MyDrive/PHI/ToF_ML/src


In [4]:
from setup import data_setup

In [6]:
%%time 
dg = data_setup()

 CPU times: user 2 s, sys: 57 ms, total: 2.05 s
Wall time: 2.07 s


In [12]:
import numba
from ast import literal_eval

In [28]:
dg.df()

,file_name,Mass/Time,MassOffset,StartFlightTime,SpecBinSize,channels,intensities
0,0106301.cas,0.387425,-0.275047,0.0,1.248,"[2644.0367300000003, 3505.0183700000002, 4162....","[73874, 1234, 138, 610, 1216, 4159, 8958, 1084..."
1,0107316.cas,0.387337,-0.278302,0.0,1.248,"[2647.00072, 3508.9949100000003, 4164.59326000...","[49864, 1034, 168, 4696, 8247, 13992, 17903, 2..."
2,0110203.cas,0.378938,-0.271015,0.0,4.992,"[1973.87665, 2049.0706800000003, 2122.01224, 2...","[23352, 74717, 10387, 947, 12344, 9121, 249, 4..."
3,0110212.cas,0.379297,-0.269824,0.0,4.992,"[672.00298, 891.90543, 1970.94521, 2046.11295,...","[34398, 304, 2223, 3521, 5205, 6509, 99, 115, ..."
4,0116511.cas,0.383481,-0.302161,0.0,1.248,"[2726.98153, 3594.53649, 4265.28736, 7866.5038...","[42995, 602, 151, 17912, 9536, 17609, 29604, 5..."
...,...,...,...,...,...,...,...
1444,PET 60keV Bi3++ Neg-41-028.cas,0.382970,-0.690830,0.0,0.138,"[32055.99122, 39923.96827, 78608.5009, 81305.4...","[31288.0, 381.0, 2023.0, 2785.0, 214.0, 4359.0..."
1445,Sample UZ5 Neg-025.cas,0.379100,-1.124670,0.0,0.128,"[43737.37474, 44200.96396, 52417.60913, 52578....","[9938.0, 80833.0, 182.0, 61.0, 158.0, 281.0, 2..."
1446,Coated2-002.cas,0.384710,-1.215490,0.0,0.128,"[45067.99069, 45217.70799, 45230.02814, 45248....","[1164697.0, 163.0, 359.0, 139.0, 152.0, 551.0,..."
1447,Sample #8 Mosaic Pos-001.cas,0.378790,-1.123260,0.0,0.128,"[43882.06046, 44228.2289, 94624.49668, 97564.5...","[71158.0, 517.0, 14355.0, 18705.0, 947.0, 5987..."


In [87]:
from data_transformation import generate_calibrated_data, mass_formula

## Timing and Making Generate Calibrated Data more efficient

In [ ]:
def original_mass_formula(channel, spec_bin_size, start_time,  mass_over_time,
                          mass_offset):
    '''
    Apply formula for calculating mass at a channel.
    '''
    return ((channel * .001 * spec_bin_size + start_time) * mass_over_time +
            mass_offset)**2

In [ ]:
def original_generate_calibrated_data(data, slope_index=4):
    '''
    Applies mass_formula to every row in dataset to allow
    calibrated graphs to be generated.
    '''
    new_data = data.copy()
    masses = []
    for row in new_data.itertuples():
        mass = []
        spec = row.SpecBinSize
        m_over_t = row[slope_index]
        m_offset = row.MassOffset
        time = row.StartFlightTime
        for tup in row.channels:
            mass.append(original_mass_formula(tup, spec, time, m_over_t,
                                              m_offset))
        masses.append(mass)
    new_data['masses'] = masses
    return new_data

In [70]:
# old method of converting each channel 1 at a time is very slow
%%time
_ = original_generate_calibrated_data(dg.df(), slope_index=2)

CPU times: user 186 ms, sys: 9.66 ms, total: 195 ms
Wall time: 197 ms


,file_name,Mass/Time,MassOffset,StartFlightTime,SpecBinSize,channels,intensities,masses
0,0106301.cas,0.387425,-0.275047,0.0,1.248,"[2644.0367300000003, 3505.0183700000002, 4162....","[73874, 1234, 138, 610, 1216, 4159, 8958, 1084...","[1.0067357827001502, 2.0154136848060986, 3.019..."
1,0107316.cas,0.387337,-0.278302,0.0,1.248,"[2647.00072, 3508.9949100000003, 4164.59326000...","[49864, 1034, 168, 4696, 8247, 13992, 17903, 2...","[1.0025002148906825, 2.0105384152423085, 3.009..."
2,0110203.cas,0.378938,-0.271015,0.0,4.992,"[1973.87665, 2049.0706800000003, 2122.01224, 2...","[23352, 74717, 10387, 947, 12344, 9121, 249, 4...","[11.9915465622805, 12.996908473240481, 14.0108..."
3,0110212.cas,0.379297,-0.269824,0.0,4.992,"[672.00298, 891.90543, 1970.94521, 2046.11295,...","[34398, 304, 2223, 3521, 5205, 6509, 99, 115, ...","[1.0051638174978792, 2.0134261084051888, 11.98..."
4,0116511.cas,0.383481,-0.302161,0.0,1.248,"[2726.98153, 3594.53649, 4265.28736, 7866.5038...","[42995, 602, 151, 17912, 9536, 17609, 29604, 5...","[1.0058658675679875, 2.011083088854326, 3.0245..."
...,...,...,...,...,...,...,...,...
1444,PET 60keV Bi3++ Neg-41-028.cas,0.382970,-0.690830,0.0,0.138,"[32055.99122, 39923.96827, 78608.5009, 81305.4...","[31288.0, 381.0, 2023.0, 2785.0, 214.0, 4359.0...","[1.0066603495617015, 2.01397574883523, 11.9966..."
1445,Sample UZ5 Neg-025.cas,0.379100,-1.124670,0.0,0.128,"[43737.37474, 44200.96396, 52417.60913, 52578....","[9938.0, 80833.0, 182.0, 61.0, 158.0, 281.0, 2...","[0.9953601182153884, 1.0407528192779132, 2.013..."
1446,Coated2-002.cas,0.384710,-1.215490,0.0,0.128,"[45067.99069, 45217.70799, 45230.02814, 45248....","[1164697.0, 163.0, 359.0, 139.0, 152.0, 551.0,...","[1.0075896611260695, 1.0224448862125326, 1.023..."
1447,Sample #8 Mosaic Pos-001.cas,0.378790,-1.123260,0.0,0.128,"[43882.06046, 44228.2289, 94624.49668, 97564.5...","[71158.0, 517.0, 14355.0, 18705.0, 947.0, 5987...","[1.0087530049036761, 1.0427493353999562, 12.00..."


In [136]:
def mass_formula(channels: np.array, spec_bin_size, start_time, mass_over_time,
                 mass_offset):
    '''
    Fast conversion from flightime to mass using numpy arrays.
    '''
    return ((channels * .001 * spec_bin_size + start_time) * 
            mass_over_time + mass_offset)**2


@njit
def numba_mass_formula(channels: np.array, spec_bin_size, start_time,
                       mass_over_time, mass_offset):
    '''
    Fast conversion from flightime to mass using numpy arrays. Uses numba.njit
    to speed up computation. so run empty version of any function using this
    before the real run to compile in. E.g. to compile generate calibrated data:
    generate_calibrated_data(pd.DataFrame(), 2, numba=True)
    njit.
    '''
    return ((channels * .001 * spec_bin_size + start_time) * mass_over_time +
            mass_offset)**2

In [133]:
def generate_calibrated_data(data, slope_index=4, numba=False):
    '''
    Applies mass_formula to every row in dataset to allow
    calibrated graphs to be generated.
    '''
    new_data = data.copy()
    masses = []
    for row in new_data.itertuples():
        spec = row.SpecBinSize
        m_over_t = row[slope_index]
        m_offset = row.MassOffset
        time = row.StartFlightTime
        if not numba:
            masses.append(mass_formula(np.array(row.channels), spec, time,
                          m_over_t, m_offset))
        else:
            masses.append(numba_mass_formula(np.array(row.channels), spec,
                          time, m_over_t, m_offset))
    new_data['masses'] = masses
    return new_data

In [137]:
# using arrays to do all conversion at once boosts speed a lot
%%time
_ = generate_calibrated_data(dg.df(), slope_index=2, numba=False)

CPU times: user 43.1 ms, sys: 890 µs, total: 43.9 ms
Wall time: 45.9 ms


In [138]:
# uncompiled numba njit run is incredibly slow
%%time
_ = generate_calibrated_data(dg.df(), slope_index=2, numba=True)

CPU times: user 388 ms, sys: 4.5 ms, total: 393 ms
Wall time: 400 ms


In [139]:
# once compiled with numba and njit this function is really fast
%%time
_ = generate_calibrated_data(dg.df(), slope_index=2, numba=True)

CPU times: user 28.9 ms, sys: 973 µs, total: 29.9 ms
Wall time: 33.4 ms


## Timing and Making Get Hydrocarbs more efficient

In [278]:
def original_get_hydrocarbs(max_num_carbs):
    '''
    Return list of hydrocarbon masses in amu by calculating all possible
    hydrocarbons with at most max_num_carbs carbons.
    '''
    carbon = 12
    hydrogen = 1.00782
    hydrocarbs = []
    for c in range(1, max_num_carbs + 1):
        hydrocarbs.append(c * carbon + (2 * c + 1) * hydrogen)
    expanded_hydrocarbs = []
    for i, hydrocarb in enumerate(hydrocarbs):
        expanded_hydrocarbs.append(hydrocarb)
        if i < len(hydrocarbs) - 1:
            prev = hydrocarb
            diff = .01564
            dist = int(hydrocarbs[i+1]) - int(hydrocarb) - 1
            for num in range(dist):
                mass = prev + 1 + diff / dist
                expanded_hydrocarbs.append(mass)
                prev = mass
    return expanded_hydrocarbs

In [279]:
# Original implementation is very slow
%%time
old = original_get_hydrocarbs(52)

CPU times: user 197 µs, sys: 15 µs, total: 212 µs
Wall time: 217 µs


In [291]:
# Rewrite hydrocarbs to work with numba, meaning without lists
def get_hydrocarbs(max_num_carbs):
    '''
    Return list of hydrocarbon masses in amu by calculating all possible
    hydrocarbons with at most max_num_carbs carbons.
    '''
    carbon = 12
    hydrogen = 1.00782
    hydrocarbs = np.zeros(max_num_carbs)
    i = 0
    for c in range(1, max_num_carbs + 1):
        hydrocarbs[i] = (c * 12 + (2 * c + 1) * 1.00782)
        i += 1

    expanded_hydrocarbs = np.zeros(int(hydrocarbs[-2]))
    i = 0
    j = 0
    for hydrocarb in hydrocarbs:
        expanded_hydrocarbs[j] = (hydrocarb)
        j += 1
        if i < len(hydrocarbs) - 1:
            prev = hydrocarb
            diff = .01564
            dist = int(hydrocarbs[i+1]) - int(hydrocarb) - 1
            for num in range(dist):
                mass = prev + 1 + diff / dist
                expanded_hydrocarbs[j] = (mass)
                j += 1
                prev = mass
        i += 1
    return expanded_hydrocarbs

In [292]:
@njit
def get_hydrocarbs_numba(max_num_carbs):
    '''
    Return list of hydrocarbon masses in amu by calculating all possible
    hydrocarbons with at most max_num_carbs carbons. Uses numba.njit
    to speed up additional runs, run get_hydrocarbs_numba(0) to compile the
    program so that numba can speed it up.
    '''
    carbon = 12
    hydrogen = 1.00782
    hydrocarbs = np.zeros(max_num_carbs)
    i = 0
    for c in range(1, max_num_carbs + 1):
        hydrocarbs[i] = (c * 12 + (2 * c + 1) * 1.00782)
        i += 1

    expanded_hydrocarbs = np.zeros(int(hydrocarbs[-2]))
    i = 0
    j = 0
    for hydrocarb in hydrocarbs:
        expanded_hydrocarbs[j] = (hydrocarb)
        j += 1
        if i < len(hydrocarbs) - 1:
            prev = hydrocarb
            diff = .01564
            dist = int(hydrocarbs[i+1]) - int(hydrocarb) - 1
            for num in range(dist):
                mass = prev + 1 + diff / dist
                expanded_hydrocarbs[j] = (mass)
                j += 1
                prev = mass
        i += 1
    return expanded_hydrocarbs


In [293]:
# Rewritten get_hydrocarbs works with numba and is very fast without it
%%time
_ = get_hydrocarbs(51)

CPU times: user 614 µs, sys: 0 ns, total: 614 µs
Wall time: 607 µs


In [294]:
# With numba compiling rewritten get_hydrocarbs is slow
%%time
new = get_hydrocarbs_numba(51)

CPU times: user 268 ms, sys: 6.71 ms, total: 274 ms
Wall time: 277 ms


In [295]:
# After numba has compiled the new rewritten get_hydrocarbs its incredibly fast
%%time
new = get_hydrocarbs_numba(51)

CPU times: user 25 µs, sys: 2 µs, total: 27 µs
Wall time: 31.9 µs


## Get Frags Dists

In [350]:
from data_transformation import get_frags_dists